# Caclulate tree visibility statistics per district

[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ac-willeke/urban-climate/blob/main/notebooks/01_FROST_extract_climate_data.ipynb) [![github](https://img.shields.io/badge/GitHub-View%20on%20GitHub-blue?logo=github)](https://github.com/ac-willeke/)

**Author**: Willeke A'Campo

**Description:** This notebooks shows how to calculate the Ecosystem Service statistics for tree visibility and impact per district using DuckDB. The results are stored in a new table in the database and exported to GeoJSON.

**Documentation:** 

### Data conversion | GeoJSON to GeoParquet   

In [1]:
import geopandas as gpd
from shapely.geometry import Point
from shapely.wkb import loads
import pyarrow
import os
import leafmap
import os
import duckdb
import pandas as pd

# TODO move to kedro pipeline
municipality = "oslo"
raw_dir = r"/workspaces/urban-climate/data/01_raw"
interim_dir = r"/workspaces/urban-climate/data/02_intermediate"
reporting_dir = r"/workspaces/urban-climate/data/08_reporting/"

# Define the table names
file_names = [
    f"{municipality}_study_area", 
    f"{municipality}_districts",
    f"{municipality}_bldg",
    f"{municipality}_res_bldg",
    f"{municipality}_green_space",
    f"{municipality}_open_space",
    f"{municipality}_public_open_space",
    f"{municipality}_private_open_space",
    f"{municipality}_tree_crowns"
    ]

table_names = [
    "study_area", "districts", "bldg", "res_bldg", "green_space",
    "open_space", "public_open_space", "private_open_space", "tree_crowns"
    ]


/usr/local/lib/python3.10/dist-packages/numpy/lib/scimath.py:46: RuntimeWarning: divide by zero encountered in log
  _ln2 = nx.log(2.0)


In [2]:

# Define the parquet_dict
parquet_dict = {
    name: os.path.join(interim_dir, f"{name}.parquet") 
    for name in file_names}

# Check if the parquet files exist, if not convert  to parquet
for key in parquet_dict.keys():
    if os.path.exists(parquet_dict[key]):
        continue
    else:
        # Define the gdf_dict
        gdf_dict = {
            name: gpd.read_file(os.path.join(raw_dir, f"{name}.geojson")) 
            for name in file_names}

        # Convert GeoDataFrame to Parquet
        for key, gdf in gdf_dict.items():
            gdf.to_parquet(
                path = interim_dir + "/" + key + ".parquet",
                index = None, 
                compression = "snappy"
            )

In [3]:
#gdf = gpd.read_file(os.path.join(raw_dir, "shp", f"{municipality}_public_open_space.shp"))
#gdf.to_parquet(
    #path = interim_dir + "/" + f"{municipality}_public_open_space" + ".parquet",
    #index = None, 
    #compression = "snappy"
#)

In [4]:
# Create a connection to the DuckDB database
con = duckdb.connect(database=":memory:", read_only=False)
con.install_extension("spatial")
con.load_extension("spatial")

# Create a table for each parquet file or GeoDataFrame
for key,table in zip(parquet_dict.keys(), table_names):
    con.execute(
        f"""
        CREATE TABLE {table} 
        AS SELECT *, ST_GeomFromWKB(geometry) 
        FROM parquet_scan('{parquet_dict[key]}')
        """
        )
    

# Fetch and print all table names
result = con.execute(
    """
    SELECT table_name 
    FROM information_schema.tables 
    WHERE table_schema = 'main'
    """
    )

print(result.fetchall())

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

[('study_area',), ('districts',), ('bldg',), ('res_bldg',), ('green_space',), ('open_space',), ('public_open_space',), ('private_open_space',), ('tree_crowns',)]


**Create a new Table with Tree Crown Center Points**

In [5]:
# Check if the 'crowns' table exists
table_exists = "tree_crowns" in [
    row[0]
    for row in con.execute(
        """
        SELECT table_name 
        FROM information_schema.tables 
        WHERE table_schema = 'main'
        """
    ).fetchall()
]

if table_exists:
    # convert dtype to DuckDB GEOMETRY
    result = con.execute(
        """
        SELECT 
        ST_X(ST_Centroid(ST_GeomFromWKB(geometry))), 
        ST_Y(ST_Centroid(ST_GeomFromWKB(geometry))) 
        FROM tree_crowns"""
        )
    # xy_crowns to pd
    df = pd.DataFrame(result.fetchall(), columns=["X", "Y"])
    
    xy_crowns = gpd.GeoDataFrame(
        df,
        geometry= gpd.points_from_xy(df.X, df.Y)
        )
    xy_crowns.crs = "EPSG:25832"


    # Create a new table in DuckDB
    con.execute(
        """
        CREATE TABLE tree_crowns_xy AS
        SELECT 
        ST_X(ST_Centroid(ST_GeomFromWKB(geometry))) AS X, 
        ST_Y(ST_Centroid(ST_GeomFromWKB(geometry))) AS Y,
        ST_Point(ST_X(ST_Centroid(ST_GeomFromWKB(geometry))), ST_Y(ST_Centroid(ST_GeomFromWKB(geometry)))) AS geometry
        FROM tree_crowns"""
    )

**Map study area and Tree Crown Center Points (10%-sample)**

In [6]:
# add layers to gdf for mapping
gdf_study_area = leafmap.read_parquet(
    parquet_dict[f"{municipality}_study_area"], 
    return_type='gdf', 
    src_crs="EPSG:25832", 
    dst_crs="EPSG:4326"
    )

# convert xy_crowns to wgs84
xy_crowns_sample = xy_crowns.sample(frac=0.05)
trees_xy = xy_crowns_sample.to_crs("EPSG:4326")
points_geojson = trees_xy.__geo_interface__
print("Map the tree crown center points (10% sample).")
print(trees_xy.head(2))

# Calculate the center of the study_area GeoDataFrame
center = gdf_study_area.geometry.unary_union.centroid

# --------------------------------------------------
# INIT MAP
# --------------------------------------------------
map = leafmap.Map()
# center
map.set_center(center.x, center.y, zoom=13)
# add Basemap
map.add_basemap("CartoDB.Positron")

# add study area as vector layer
map.add_gdf(
        gdf_study_area, 
        layer_name="study_area", 
        get_fill_color=[0, 0, 255, 128]
        )

map.add_gdf(
    trees_xy,
    layer_name ="trees",
    color= "black"
)

#map

Map the tree crown center points (10% sample).
                    X             Y                   geometry
349265  607147.627212  6.646218e+06  POINT (10.91765 59.93943)
58924   597707.327961  6.643856e+06  POINT (10.74768 59.92058)


### Database Connection | DuckDB

In [7]:
%%capture
columns = con.execute("PRAGMA table_info(districts)").fetchall()
for column in columns:
    print(column[1])


**Split open space, buildings and tree crowns by district**

In [8]:
%%capture
# create new table split_open_space 
# with open space split by district boundaries
con.execute(
    """
    CREATE TABLE split_open_space AS 
    SELECT
        districts.grunnkretsnummer,
        ST_Intersection(ST_GeomFromWKB(districts.geometry), ST_GeomFromWKB(open_space.geometry)) AS geom
    FROM 
        districts, open_space
    WHERE
        ST_Intersects(ST_GeomFromWKB(districts.geometry), ST_GeomFromWKB(open_space.geometry));
    """
    )

: 

In [ ]:
%%capture
# create new table split_private_space
# with public space split by district boundaries
con.execute(
    """
    CREATE TABLE split_private_space AS 
    SELECT
        districts.grunnkretsnummer,
        ST_Intersection(ST_GeomFromWKB(districts.geometry), ST_GeomFromWKB(private_open_space.geometry)) AS geom
    FROM 
        districts, private_open_space
    WHERE
        ST_Intersects(ST_GeomFromWKB(districts.geometry), ST_GeomFromWKB(private_open_space.geometry));
    """
    )

In [ ]:
%%capture
# create new table split_public_space
# with private space split by district boundaries
con.execute(
    """
    CREATE TABLE split_public_space AS 
    SELECT
        districts.grunnkretsnummer,
        ST_Intersection(ST_GeomFromWKB(districts.geometry), ST_GeomFromWKB(public_open_space.geometry)) AS geom
    FROM 
        districts, public_open_space
    WHERE
        ST_Intersects(ST_GeomFromWKB(districts.geometry), ST_GeomFromWKB(public_open_space.geometry));
    """
    )

In [ ]:
%%capture
# create new table split_buildings
# with buildings split by district boundaries
con.execute(
    """
    CREATE TABLE split_bldg AS 
    SELECT
        districts.grunnkretsnummer,
        ST_Intersection(ST_GeomFromWKB(districts.geometry), ST_GeomFromWKB(bldg.geometry)) AS geom
    FROM 
        districts, bldg
    WHERE
        ST_Intersects(ST_GeomFromWKB(districts.geometry), ST_GeomFromWKB(bldg.geometry));
    """
    )

In [ ]:
%%capture
# create new table split_res_bldg
# with buildings split by district boundaries
con.execute(
    """
    CREATE TABLE split_res_bldg AS 
    SELECT
        districts.grunnkretsnummer,
        ST_Intersection(ST_GeomFromWKB(districts.geometry), ST_GeomFromWKB(res_bldg.geometry)) AS geom
    FROM 
        districts, res_bldg
    WHERE
        ST_Intersects(ST_GeomFromWKB(districts.geometry), ST_GeomFromWKB(res_bldg.geometry));
    """
    )

In [ ]:
%%capture
# create new table split_tree_crowns
# with tree crowns split by district boundaries
con.execute(
    """
    CREATE TABLE split_tree_crowns AS 
    SELECT
        districts.grunnkretsnummer,
        ST_Intersection(ST_GeomFromWKB(districts.geometry), ST_GeomFromWKB(tree_crowns.geometry)) AS geom
    FROM 
        districts, tree_crowns
    WHERE
        ST_Intersects(ST_GeomFromWKB(districts.geometry), ST_GeomFromWKB(tree_crowns.geometry));
    """
    )

In [ ]:
%%capture
# print columns split_open_space
columns = con.execute("PRAGMA table_info(split_open_space)").fetchall()
for column in columns:
    print(column[1])
    
# Query to fetch all table names
tables = con.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()

# Print the first row of each table
for table in tables:
    first_row = con.execute(f"SELECT * FROM {table[0]} LIMIT 1").fetchone()
    print(f"First row of {table[0]}: {first_row}")

### Generate Columns with Count Statistics

| Name | Alias | Description | Type |  Unit | 
| --- | --- | --- | --- | --- |
| n_trees | Antall trær | Number of trees in the district | INT |
| n_bldg | Antall bygninger | Number of buildings in the district | INT |
| n_res_bldg | Antall boliger | Number of residential buildings in the district | INT |
| n_res_bldg_near_gs | Antall boliger nær grøntområde (300 m) | Number of residential buildings near green space (300 m) | INT |
| n_trees_near_rbldg | Antall trær nær boliger (15 m) | Number of trees near residential buildings (15 m) | INT |
| n_viewshed | Antall viewshed piksler | Number of viewshed pixels that intersect with the building edge | INT |



**Add columns to districts table**

In [ ]:
# update districts
# add count columns: n_trees, n_bldg, n_res_bldg, n_res_bldg_near_gs, n_trees_near_rbldg
con.execute("ALTER TABLE districts ADD COLUMN n_trees INTEGER")
con.execute("ALTER TABLE districts ADD COLUMN n_bldg INTEGER")
con.execute("ALTER TABLE districts ADD COLUMN n_res_bldg INTEGER")
con.execute("ALTER TABLE districts ADD COLUMN n_res_bldg_near_gs INTEGER")
con.execute("ALTER TABLE districts ADD COLUMN perc_near_gs INTEGER")
con.execute("ALTER TABLE districts ADD COLUMN n_trees_near_rbldg INTEGER")
con.execute("ALTER TABLE districts ADD COLUMN perc_near_rbldg INTEGER")

In [ ]:
%%capture
# print columns districts
columns = con.execute("PRAGMA table_info(split_bldg)").fetchall()
for column in columns:
    print(column[1])

**Calculate COUNT attributes**

In [ ]:
# COUNT number of trees per district
n_trees = con.execute(
    """
    SELECT districts.grunnkretsnummer, COUNT(*) 
    FROM tree_crowns_xy 
    JOIN districts ON ST_Within(tree_crowns_xy.geometry, ST_GeomFromWKB(districts.geometry))
    GROUP BY districts.grunnkretsnummer
    """
    ).fetchall()

# COUNT number of buildings per district
n_bldg = con.execute(
    """
    SELECT districts.grunnkretsnummer, COUNT(*) 
    FROM split_bldg 
    JOIN districts ON ST_Within(split_bldg.geom, ST_GeomFromWKB(districts.geometry))
    GROUP BY districts.grunnkretsnummer
    """
    ).fetchall()

# COUNT number of residential buildings per district 
n_res_bldg = con.execute(
    """
    SELECT districts.grunnkretsnummer, COUNT(*) 
    FROM split_res_bldg 
    JOIN districts ON ST_Within(split_res_bldg.geom, ST_GeomFromWKB(districts.geometry))
    GROUP BY districts.grunnkretsnummer
    """
    ).fetchall()

# COUNT number of res buildings WITHIN 300m distance of green space
n_res_bldg_near_gs = con.execute(
    """
    SELECT districts.grunnkretsnummer, COUNT(*) 
    FROM res_bldg 
    JOIN districts ON ST_Within(ST_GeomFromWKB(res_bldg.geometry), ST_GeomFromWKB(districts.geometry))
    WHERE res_bldg.geometry IS NOT NULL AND districts.geometry IS NOT NULL AND EXISTS (
        SELECT 1
        FROM green_space
        WHERE green_space.geometry IS NOT NULL AND ST_DWithin(ST_GeomFromWKB(res_bldg.geometry), ST_GeomFromWKB(green_space.geometry), 300)
    )
    GROUP BY districts.grunnkretsnummer
    """
    ).fetchall()

# COUNT number of trees crowns WITHIN 15m distance of res buildings
n_trees_near_rbldg = con.execute(
    """
    SELECT districts.grunnkretsnummer, COUNT(*) 
    FROM tree_crowns
    JOIN districts ON ST_Within(ST_GeomFromWKB(tree_crowns.geometry), ST_GeomFromWKB(districts.geometry))
    WHERE EXISTS (
        SELECT 1
        FROM res_bldg
        WHERE ST_DWithin(ST_GeomFromWKB(tree_crowns.geometry), ST_GeomFromWKB(res_bldg.geometry), 15)
        AND ST_Within(ST_GeomFromWKB(res_bldg.geometry), ST_GeomFromWKB(districts.geometry))
    )
    GROUP BY districts.grunnkretsnummer
    """
    ).fetchall()

# Update districts table
for id, count in n_trees:
    con.execute(f"UPDATE districts SET n_trees = {count} WHERE grunnkretsnummer = {id}")

for id, count in n_bldg:
    con.execute(f"UPDATE districts SET n_bldg = {count} WHERE grunnkretsnummer = {id}")

for id, count in n_res_bldg:
    con.execute(f"UPDATE districts SET n_res_bldg = {count} WHERE grunnkretsnummer = {id}")

for id, count in n_res_bldg_near_gs:
    con.execute(f"UPDATE districts SET n_res_bldg_near_gs = {count} WHERE grunnkretsnummer = {id}")

for id, count in n_trees_near_rbldg:
    con.execute(f"UPDATE districts SET n_trees_near_rbldg = {count} WHERE grunnkretsnummer = {id}")
    
# print columns districts
columns = con.execute("PRAGMA table_info(districts)").fetchall()
for column in columns:
    print(column[1])

In [ ]:
# if NAN set to 0
con.execute(
    """
    UPDATE districts
    SET n_res_bldg_near_gs = COALESCE(n_res_bldg_near_gs, 0)
    """
)

con.execute(
    """
    UPDATE districts
    SET n_bldg = COALESCE(n_bldg, 0)
    """
)

con.execute(
    """
    UPDATE districts
    SET n_res_bldg = COALESCE(n_res_bldg, 0)
    """
)

con.execute(
    """
    UPDATE districts
    SET n_trees_near_rbldg = COALESCE(n_trees_near_rbldg, 0)
    """
)
# normalize perc_near_gs
con.execute(
    """
    UPDATE districts
    SET perc_near_gs = (n_res_bldg_near_gs / n_res_bldg) * 100
    """
)

# normalize perc_near_rbldg
con.execute(
    """
    UPDATE districts
    SET perc_near_rbldg = (n_trees_near_rbldg / n_trees) * 100
    """
)

### Generate Columns with Area Statistics

| Name | Alias | Description | Type |  Unit |
| --- | --- | --- | --- | --- |
| a_district | Grunnkretsareal | Area of the district | FLOAT | m2 |
| a_open_space | Åpent område | Area of open space | FLOAT | m2 |
| a_private_space | Privat område | Area of private space | FLOAT | m2 |
| a_public_space | Offentlig område | Area of public space | FLOAT | m2 |
| a_crown | Kroneareal | Crown coverage area within the district | FLOAT | m2 |
| perc_crown | Trekronedekningsgrad (%) | Tree crown coverage % | FLOAT | % |


**Add columns to districts table**

In [ ]:
# update districts
# add area columns: a_open_space, a_private_space, a_public_space, a_green_space, a_crown, a_crown_public, a_crown_private
con.execute("ALTER TABLE districts ADD COLUMN a_open_space FLOAT")
con.execute("ALTER TABLE districts ADD COLUMN a_private_space FLOAT")
con.execute("ALTER TABLE districts ADD COLUMN a_public_space FLOAT")
con.execute("ALTER TABLE districts ADD COLUMN a_crown FLOAT")
con.execute("ALTER TABLE districts ADD COLUMN perc_crown FLOAT")

**Caluclate AREA attributes**

In [ ]:
# AREA of open space per district
# calculate the overlapping area of open space with district X, A etc. 
con.execute(
    """
    UPDATE districts
    SET a_open_space = (
        SELECT SUM(ST_Area(ST_Intersection(ST_GeomFromWKB(districts.geometry), split_open_space.geom)))
        FROM split_open_space
        WHERE ST_Intersects(ST_GeomFromWKB(districts.geometry), split_open_space.geom)
    )
    """
)

# AREA of private open space per district
# calculate the overlapping area of private open space with district X, A etc.
con.execute(
    """
    UPDATE districts
    SET a_private_space = (
        SELECT SUM(ST_Area(ST_Intersection(ST_GeomFromWKB(districts.geometry), split_private_space.geom)))
        FROM split_private_space
        WHERE ST_Intersects(ST_GeomFromWKB(districts.geometry), split_private_space.geom)
    )
    """
)

# AREA of public open space per district
# calculate the overlapping area of public open space with district X, A etc.
con.execute(
    """
    UPDATE districts
    SET a_public_space = (
        SELECT SUM(ST_Area(ST_Intersection(ST_GeomFromWKB(districts.geometry), split_public_space.geom)))
        FROM split_public_space
        WHERE ST_Intersects(ST_GeomFromWKB(districts.geometry), split_public_space.geom)
    )
    """
)

# AREA of tree crowns per district
# calculate the overlapping area of tree crowns with district X, A etc.
con.execute(
    """
    UPDATE districts
    SET a_crown = (
        SELECT SUM(ST_Area(ST_Intersection(ST_GeomFromWKB(districts.geometry), split_tree_crowns.geom)))
        FROM split_tree_crowns
        WHERE ST_Intersects(ST_GeomFromWKB(districts.geometry), split_tree_crowns.geom)
    )
    """
)

# PERCENTAGE of tree crown coverage per district 
con.execute(
    """
    UPDATE districts
    SET perc_crown = (a_crown / a_clipped) * 100
    """
)

**Export to dataframe**

In [ ]:
# Export districts to DataFrame
df = pd.read_sql("SELECT * FROM districts", con)

# Convert geometry column from WKB to shapely geometry
df['geometry'] = df['geometry'].apply(loads, hex=True)

# Convert DataFrame to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry')
gdf_sorted = gdf.sort_values(by='grunnkretsnummer', ascending=True)
gdf_sorted = gdf_sorted.round(2)

# Define the %-bins and labels
labels = ["no data", "0-25%", "25-50%", "50-75%", "75-100%"]
bins = pd.IntervalIndex.from_tuples([(-0.01, 25), (25, 50), (50, 75), (75, 100)])
dict = {
    "nan":"no data", 
    "(-0.01, 25.0]": "0-25%", 
    "(25.0, 50.0]": "25-50%", 
    "(50.0, 75.0]": "50-75%", 
    "(75.0, 100.0]": "75-100%"
    }

# Near Residential Buildings % Categories
gdf_sorted['bldg_bins'] = pd.cut(gdf_sorted['perc_near_rbldg'], bins)
gdf_sorted['bldg_bins'] = gdf_sorted['bldg_bins'].astype(str)
gdf_sorted['labels_near_rbldg'] = gdf_sorted['bldg_bins'].map(dict)

# Near Green Space % Categories
gdf_sorted['gs_bins'] = pd.cut(gdf_sorted['perc_near_gs'], bins)
gdf_sorted['gs_bins'] = gdf_sorted['gs_bins'].astype(str)
gdf_sorted['labels_near_gs'] = gdf_sorted['gs_bins'].map(dict)

# Crown Coverate % Categories
gdf_sorted['crown_bins'] = pd.cut(gdf_sorted['perc_crown'], bins)
gdf_sorted['crown_bins'] = gdf_sorted['crown_bins'].astype(str)
gdf_sorted['labels_perc_crown'] = gdf_sorted['crown_bins'].map(dict)

gdf_sorted.drop(columns=['bldg_bins', 'gs_bins', 'crown_bins'], inplace=True)

# display col "kommunenavn","grunnkrestnummer", "grunnkretsnavn" n_bldg, n_res_bldg, n_res_bldg_near_gs, n_trees_near_rbldg, a_open_space, a_private_space, a_public_space, a_crown, perc_crown
display(gdf_sorted[[
    "kommunenavn","grunnkretsnummer", "grunnkretsnavn", 
    "n_trees", "n_bldg", "n_res_bldg", 
    "n_res_bldg_near_gs", "perc_near_gs","labels_near_gs",
    "n_trees_near_rbldg", "perc_near_rbldg", "labels_near_rbldg",
    "a_open_space", "a_private_space", "a_public_space", "a_crown", 
    "perc_crown", "labels_perc_crown"
    ]]
        )
gdf.crs = "EPSG:25832"
gdf_sorted.crs = "EPSG:25832"


gdf_mapping = gdf.to_crs("EPSG:4326")

In [ ]:
%%capture
print(gdf_sorted.columns)
#drop duckdb geom col
gdf_sorted.drop(columns=['st_geomfromwkb(geometry)'], inplace=True)

# sort columns 
new_order = ['OBJECTID', 'fylkesnummer', 'fylkesnavn', 'kommunenummer',
       'kommunenavn', 'delomradenummer', 'delomradenavn', 'grunnkretsnummer',
       'grunnkretsnavn', 'kilde_admin', 'kilde_befolkning', 'id_befolkning',
       'year_pop_stat', 'pop_total', 'pop_elderly', 'a_district', 'a_unit',
       'a_clipped', 'n_trees', 'n_bldg', 'n_res_bldg',
       'n_res_bldg_near_gs', 'perc_near_gs', 'labels_near_gs',
       'n_trees_near_rbldg', 'perc_near_rbldg', 'labels_near_rbldg',
       'a_open_space', 'a_private_space', 'a_public_space',
       'a_crown', 'perc_crown','labels_perc_crown',  
       'SHAPE_Length', 'SHAPE_Area', 'geometry']

# Reorder the columns
gdf_sorted = gdf_sorted[new_order]
print(gdf_sorted.columns)

**Export to Parquet, GeoJSON and CSV format**

In [ ]:
# Export GDF to file 
filepath = os.path.join(reporting_dir, f"{municipality}_district_treeVis_stat")

# Write to .parquet
gdf_sorted.to_parquet(os.path.join(filepath + '.parquet'))

# Write to .geojson
gdf_sorted.to_file(os.path.join(filepath + '.geojson'), driver='GeoJSON')

# Write to .csv
gdf_sorted.to_csv(os.path.join(filepath + '.csv'))